In [13]:
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from torch.utils.data import DataLoader
# from torchvision import transforms
import torch.nn as nn
from timeit import Timer
from tqdm import tqdm
import pandas as pd
from pandas import DataFrame as df

In [26]:
df = pd.read_csv("/kaggle/input/geoelectric-models/geoelectric_models.csv")
X = df[[f'dBzdt_{i}' for i in range(1, 41)]].values
y = df[[f'rho_{i}' for i in range(1, 21)] + [f'thk_{i}' for i in range(1, 20)]].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(X.shape)
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
class NLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(40, 80),
            nn.ReLU(),
            nn.Linear(80, 40),
            nn.ReLU(),
            nn.Linear(40, 39)
        )
    
    def forward(self, x):
        return self.layers(x)

model = NLP()
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
def train_model(model,EPOCHS = 100):
    # EPOCHS = 100
    for epoch in range(EPOCHS):
        model.train()
        y_pred = model(X_train_t)
        loss = loss_fn(y_pred, y_train_t)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch  % 1000 == 0 :
            print(f"Epoch {epoch}: Loss = {loss.item():.6f}")
train_model(model, EPOCHS=10000)

Epoch 0: Loss = 380.196655
Epoch 1000: Loss = 380.196655
Epoch 2000: Loss = 380.196655
Epoch 3000: Loss = 380.196655
Epoch 4000: Loss = 380.196655
Epoch 5000: Loss = 380.196655
Epoch 6000: Loss = 380.196655
Epoch 7000: Loss = 380.196655
Epoch 8000: Loss = 380.196655
Epoch 9000: Loss = 380.196655


In [39]:
act = nn.ReLU()

class Model_comp(nn.Module):
  def __init__(self):
    super().__init__()
    self.ll1 = nn.Linear(40,128)
    self.ac1 = nn.ReLU()
    self.ll2 = nn.Linear(128,128)
    self.ac2 = nn.ReLU()
    self.ll3 = nn.Linear(128,128)
    self.ac3 = nn.ReLU()
    self.ll4 = nn.Linear(128,128)
    self.ac4 = nn.ReLU()
    self.output = nn.Linear(128,39)

  def forward(self,X):
    X = self.ll1(X)
    X = self.ac1(X)
    X = self.ll2(X)
    X = self.ac2(X)
    X = self.ll3(X)
    X = self.ac3(X)
    X = self.ll4(X)
    X = self.ac4(X)
    X = self.output(X)
    return(X)

dense_model = Model_comp()
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(dense_model.parameters(), lr=1e-3)

epochs = 10000
for epoch in range(epochs):
  dense_model.train()
  y_pred = dense_model(X_train_t)
  loss_train = loss_fn(y_pred,y_train_t)
  optimizer.zero_grad()
  loss_train.backward()
  optimizer.step()

  dense_model.eval()
  with torch.inference_mode():
    y_pred_test = dense_model(X_test_t)
    loss_test = loss_fn(y_pred_test,y_test_t)

  if epoch % 100 == 0:
    print(f"epoch:{epoch} | loss_train: {loss_train} | loss_test: {loss_test}")

epoch:0 | loss_train: 380.2528076171875 | loss_test: 381.0336608886719
epoch:100 | loss_train: 188.6985321044922 | loss_test: 189.80702209472656
epoch:200 | loss_train: 187.15835571289062 | loss_test: 188.6143341064453
epoch:300 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:400 | loss_train: 187.1582794189453 | loss_test: 188.6155242919922
epoch:500 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:600 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:700 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:800 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:900 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:1000 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:1100 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:1200 | loss_train: 187.1582794189453 | loss_test: 188.61550903320312
epoch:1400 | loss_train: 187.1582794189

In [44]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class DBzdtDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

class TransformerRegressor(nn.Module):
    def __init__(self, seq_len=40, d_model=64, n_heads=4, n_layers=3, dim_ff=128, out_dim=39, dropout=0.1):
        super().__init__()
        self.token_emb = nn.Linear(1, d_model)
        self.pos_emb   = nn.Parameter(torch.randn(1, seq_len, d_model))
        layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads, dim_feedforward=dim_ff, dropout=dropout, activation="gelu", batch_first=True)
        self.encoder = nn.TransformerEncoder(layer, num_layers=n_layers)
        self.head    = nn.Sequential(nn.LayerNorm(d_model), nn.Linear(d_model, out_dim))
    def forward(self, x):
        b, t = x.shape
        x = x.unsqueeze(-1)
        x = self.token_emb(x)
        x = x + self.pos_emb[:, :t, :]
        x = self.encoder(x)
        x = x.mean(dim=1)
        return self.head(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
train_ds = DBzdtDataset(X_train, y_train)
test_ds  = DBzdtDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=2)

model = TransformerRegressor()
model.to(device)
opt   = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=5, verbose=True)
loss_fn = nn.MSELoss()

for epoch in range(1, 101):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds  = model(xb)
        loss   = loss_fn(preds, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        running_loss += loss.item() * xb.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    sched.step(train_loss)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            val_loss += loss_fn(model(xb), yb).item() * xb.size(0)
    val_loss /= len(test_loader.dataset)
    print(f"Epoch {epoch:03d} Train MSE: {train_loss:.6f} Val MSE: {val_loss:.6f} LR: {opt.param_groups[0]['lr']:.2e}")


Epoch 001 Train MSE: 211178.673688 Val MSE: 212271.139125 LR: 1.00e-04
Epoch 002 Train MSE: 209671.966750 Val MSE: 210659.185250 LR: 1.00e-04
Epoch 003 Train MSE: 207952.227500 Val MSE: 208831.685875 LR: 1.00e-04
Epoch 004 Train MSE: 206031.879750 Val MSE: 206809.183875 LR: 1.00e-04
Epoch 005 Train MSE: 203919.715875 Val MSE: 204602.283875 LR: 1.00e-04
Epoch 006 Train MSE: 201633.045188 Val MSE: 202226.902500 LR: 1.00e-04
Epoch 007 Train MSE: 199188.704500 Val MSE: 199703.439250 LR: 1.00e-04
Epoch 008 Train MSE: 196598.860500 Val MSE: 197038.295750 LR: 1.00e-04
Epoch 009 Train MSE: 193876.802469 Val MSE: 194247.964375 LR: 1.00e-04
Epoch 010 Train MSE: 191036.221562 Val MSE: 191343.950750 LR: 1.00e-04
Epoch 011 Train MSE: 188086.801781 Val MSE: 188335.025875 LR: 1.00e-04
Epoch 012 Train MSE: 185040.475406 Val MSE: 185238.787250 LR: 1.00e-04
Epoch 013 Train MSE: 181908.148719 Val MSE: 182059.294125 LR: 1.00e-04
Epoch 014 Train MSE: 178698.974875 Val MSE: 178804.295750 LR: 1.00e-04
Epoch 

In [ ]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, skip=True):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.skip = skip
        if skip and in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, 1)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.skip:
            out += identity
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_proj = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.layer1 = ResNetBlock(32, 64, skip=True)
        self.layer2 = ResNetBlock(64, 64, skip=True)
        self.layer3 = ResNetBlock(64, 128, skip=True)
        self.layer4 = ResNetBlock(128, 128, skip=True)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, 39)

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = self.input_proj(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.global_pool(x).squeeze(-1)
        x = self.fc(x)
        return x
model = ResNet()
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10000):
    model.train()
    y_pred = model(X_train_t)
    loss = loss_fn(y_pred, y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test_t)
        loss_test = loss_fn(y_pred_test, y_test_t)
    if epoch % 100 == 0:
        print(f"epoch:{epoch} | loss_train: {loss} | loss_test: {loss_test}")


In [ ]:
def predict(X):
    model.eval()
    with torch.no_grad():
        X_t = torch.tensor(X, dtype=torch.float32)
        y_pred = model(X_t)
        return y_pred.numpy()

print(predict(X_test[:1]))